# Classification and regression models

In [1]:
import rebound
from spock import FeatureClassifier
feature_model = FeatureClassifier()

sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1., e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.move_to_com()

print(feature_model.predict_stable(sim))

0.06591137


In [2]:
from spock import DeepRegressor
deep_model = DeepRegressor()

median, lower, upper = deep_model.predict_instability_time(sim, samples=10000)
print(int(median))

231647


In [3]:
from spock import CollisionMergerClassifier
class_model = CollisionMergerClassifier()

prob_12, prob_23, prob_13 = class_model.predict_collision_probs(sim)

print('Probability planet 1 collides with 2:', prob_12)
print('Probability planet 2 collides with 3:', prob_23)
print('Probability planet 1 collides with 3:', prob_13)

Probability planet 1 collides with 2: 0.2738345
Probability planet 2 collides with 3: 0.49277353
Probability planet 1 collides with 3: 0.23339202


In [4]:
from spock import CollisionOrbitalOutcomeRegressor
reg_model = CollisionOrbitalOutcomeRegressor()

new_sim = reg_model.predict_collision_outcome(sim, collision_inds=[2, 3])

print('Predicted P1:', new_sim.particles[1].P)
print('Predicted e1:', new_sim.particles[1].e, '\n')
print('Predicted P2:', new_sim.particles[2].P)
print('Predicted e2:', new_sim.particles[2].e)

Predicted P1: 1.2250376725562897
Predicted e1: 0.058889485476765806 

Predicted P2: 1.412646240582396
Predicted e2: 0.029298374243317323


# Giant impact emulator

In [1]:
import rebound
sim = rebound.Simulation()
sim.add(m=1.)
sim.add(m=1.e-5, P=1, e=0.03, pomega=2., l=0.5)
sim.add(m=1.e-5, P=1.2, e=0.03, pomega=3., l=3.)
sim.add(m=1.e-5, P=1.5, e=0.03, pomega=1.5, l=2.)
sim.add(m=1.e-5, P=2.2, e=0.03, pomega=0.5, l=5.0)
sim.add(m=1.e-5, P=2.5, e=0.03, pomega=5.0, l=1.5)
sim.move_to_com()

In [2]:
from spock import GiantImpactPhaseEmulator
emulator = GiantImpactPhaseEmulator(seed=42)

In [7]:
%%time
pred_sim = emulator.predict(sim)

CPU times: user 1.34 s, sys: 713 ms, total: 2.05 s
Wall time: 1.1 s


In [3]:
%%time
for i in range(100):
    print(i)
    emulator = GiantImpactPhaseEmulator(seed=0)
    pred_sim = emulator.predict(sim)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
CPU times: user 1min 45s, sys: 1min 4s, total: 2min 49s
Wall time: 1min 26s


In [4]:
import einops
einops.__version__

'0.6.0'

In [8]:
ps = pred_sim.particles
print('t =', pred_sim.t, '\n')
for i in range(1, len(ps)):
    print('Predicted m' + str(i), '=', ps[i].m)
    print('Predicted P' + str(i), '=', ps[i].P)
    print('Predicted e' + str(i), '=', ps[i].e, '\n')

t = 999999999.9999985 

Predicted m1 = 3.0000000000000004e-05
Predicted P1 = 1.4789032321188267
Predicted e1 = 0.027463762835280816 

Predicted m2 = 2e-05
Predicted P2 = 3.519481719959543
Predicted e2 = 0.02516227040146231 

